# Optimización de Modelo Higgs → WW*

**Objetivo**: Mejorar el rendimiento del modelo de clasificación mediante:
- Optimización bayesiana de hiperparámetros (Optuna)
- Análisis comparativo de múltiples estrategias
- Selección automática del mejor modelo

**Dataset**: 26,277 eventos ATLAS (43% H→WW*, 57% DibosonWW)

**Enfoque**: Optimizar hiperparámetros usando las features ya seleccionadas

---

In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
import importlib
from src.features import feature_engineering
from src.models import boosting
importlib.reload(boosting)
from src.features.feature_engineering import add_feature_engineering
from src.models.boosting import ams_score
import optuna
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

plt.style.use('ggplot')
print("Entorno configurado (módulos recargados)")

Entorno configurado (módulos recargados)


---
## Cargar Datos y Features Actuales**

In [2]:
# Cargar datos
df = pd.read_pickle("../data/interim/merged_raw.pkl")
df = add_feature_engineering(df)

# Cargar features seleccionadas
with open("../models/final_features.json", "r") as f:
    current_features = json.load(f)

X = df[current_features]
y = df["target"]

print(f"Dataset: {X.shape[0]} eventos, {X.shape[1]} features")
print(f"Balance: {y.mean():.1%} Higgs, {1-y.mean():.1%} WW")


model_actual = joblib.load("../models/best_model.pkl")
y_pred = model_actual.predict_proba(X)[:,1]
auc_baseline = roc_auc_score(y, y_pred)
ams_baseline = ams_score(y, y_pred)

print(f"\nMÉTRICAS ACTUALES (Baseline):")
print(f"   AUC: {auc_baseline:.4f}")
print(f"   AMS: {ams_baseline:.4f}")

Dataset: 26277 eventos, 14 features
Balance: 43.2% Higgs, 56.8% WW

MÉTRICAS ACTUALES (Baseline):
   AUC: 0.8591
   AMS: 116.2038

MÉTRICAS ACTUALES (Baseline):
   AUC: 0.8591
   AMS: 116.2038


---
## Optimización con Optuna (Bayesian Optimization)

In [ ]:
# Función objetivo para XGBoost
def objective_xgb(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000, step=50),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'random_state': 42,
        'eval_metric': 'auc',
        'use_label_encoder': False
    }
    
    model = XGBClassifier(**params)
    
    # Cross-validation para evitar overfitting
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    return scores.mean()

print("Iniciando optimización con Optuna...")
print("Búsqueda bayesiana en espacio de 9 hiperparámetros")
print("Trials: 50 | CV: 5-fold | Tiempo estimado: 10-30 minutos\n")


optuna.logging.set_verbosity(optuna.logging.WARNING)

study_xgb = optuna.create_study(direction='maximize', study_name='xgb_higgs')
study_xgb.optimize(objective_xgb, n_trials=50, show_progress_bar=True)

print(f"\nOptimización completada")
print(f"   Mejor AUC (CV): {study_xgb.best_value:.4f}")
print(f"   Mejora vs baseline: {'+' if study_xgb.best_value > auc_baseline else ''}{(study_xgb.best_value - auc_baseline)*100:.2f}%")

Iniciando optimización con Optuna...
Búsqueda bayesiana en espacio de 9 hiperparámetros
Trials: 50 | CV: 5-fold | Tiempo estimado: 10-30 minutos



  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Visualizar optimización
fig = optuna.visualization.plot_optimization_history(study_xgb)
fig.show()

fig = optuna.visualization.plot_param_importances(study_xgb)
fig.show()

In [ ]:
# Mejores hiperparámetros
print("MEJORES HIPERPARÁMETROS:")
print("-" * 50)
for key, value in study_xgb.best_params.items():
    print(f"   {key:20s}: {value}")

🏆 MEJORES HIPERPARÁMETROS:
--------------------------------------------------
   n_estimators        : 400
   max_depth           : 5
   learning_rate       : 0.012588704596236986
   subsample           : 0.7055292042939553
   colsample_bytree    : 0.7440366233650568
   min_child_weight    : 1
   gamma               : 0.304566306192325
   reg_alpha           : 4.4054830476834914e-07
   reg_lambda          : 9.913444876084768


---
## 🎯 **Paso 3: Entrenar Modelo con Hiperparámetros Optimizados**

In [ ]:
# Entrenar modelo con mejores hiperparámetros encontrados por Optuna
best_model = XGBClassifier(**study_xgb.best_params)
best_model.fit(X, y)

# Evaluar
from sklearn.metrics import accuracy_score, f1_score

y_pred_optimized = best_model.predict_proba(X)[:,1]
y_pred_optimized_binary = (y_pred_optimized >= 0.5).astype(int)

auc_optimized = roc_auc_score(y, y_pred_optimized)
ams_optimized = ams_score(y, y_pred_optimized)
acc_optimized = accuracy_score(y, y_pred_optimized_binary)
f1_optimized = f1_score(y, y_pred_optimized_binary)

# Calcular métricas baseline
y_pred_binary = (y_pred >= 0.5).astype(int)
acc_baseline = accuracy_score(y, y_pred_binary)
f1_baseline = f1_score(y, y_pred_binary)

print("\n" + "="*60)
print("COMPARACIÓN DE RESULTADOS")
print("="*60)
print(f"\n📊 BASELINE (Hiperparámetros Originales):")
print(f"   AUC:      {auc_baseline:.4f}")
print(f"   Accuracy: {acc_baseline:.4f}")
print(f"   F1-Score: {f1_baseline:.4f}")
print(f"   AMS:      {ams_baseline:.4f}")
print(f"   Features: {len(current_features)}")

print(f"\n🚀 MODELO OPTIMIZADO (Hiperparámetros Optuna):")
print(f"   AUC:      {auc_optimized:.4f} ({'+' if auc_optimized >= auc_baseline else ''}{(auc_optimized-auc_baseline)*100:.2f}%)")
print(f"   Accuracy: {acc_optimized:.4f} ({'+' if acc_optimized >= acc_baseline else ''}{(acc_optimized-acc_baseline)*100:.2f}%)")
print(f"   F1-Score: {f1_optimized:.4f} ({'+' if f1_optimized >= f1_baseline else ''}{(f1_optimized-f1_baseline)*100:.2f}%)")
print(f"   AMS:      {ams_optimized:.4f} ({'+' if ams_optimized >= ams_baseline else ''}{(ams_optimized-ams_baseline)*100:.2f}%)")
print(f"   Features: {len(current_features)} (sin cambios)")

print("\n" + "="*60)

# Determinar ganador
if auc_optimized > auc_baseline:
    print("\n✅ RESULTADO: El modelo optimizado es MEJOR")
    print(f"   Ganancia en AUC: +{(auc_optimized-auc_baseline)*100:.2f}%")
elif auc_optimized < auc_baseline:
    print("\n⚠️ RESULTADO: El modelo baseline es MEJOR")
    print(f"   El baseline ya estaba bien calibrado")
else:
    print("\n🔄 RESULTADO: Rendimiento equivalente")
    print(f"   Ambos modelos tienen AUC similar")


COMPARACIÓN DE RESULTADOS

📊 BASELINE:
   AUC:      0.8651
   Accuracy: 0.7777
   F1-Score: 0.7520
   AMS:      117.6011
   Features: 15

🚀 MODELO MEJORADO:
   AUC:      0.7803 (+-8.47%)
   Accuracy: 0.7045 (+-7.32%)
   F1-Score: 0.6683 (+-8.37%)
   AMS:      97.8023 (+-1979.88%)
   Features: 23 (+8)



---
## 💡 **Paso 4: Interpretación de Resultados**

### 📊 **Análisis de la Optimización**

La optimización bayesiana con Optuna exploró 50 combinaciones diferentes de hiperparámetros en un espacio de 9 dimensiones:
- `n_estimators`: Número de árboles
- `max_depth`: Profundidad máxima
- `learning_rate`: Tasa de aprendizaje
- `subsample`, `colsample_bytree`: Sampling de datos y features
- `min_child_weight`, `gamma`: Regularización por complejidad
- `reg_alpha`, `reg_lambda`: Regularización L1 y L2

### ✅ **Si el modelo optimizado ganó:**
- Los hiperparámetros originales no eran óptimos para este dataset
- La optimización bayesiana encontró una mejor región del espacio de parámetros
- **Acción**: Guardar modelo optimizado y usar en producción

### ⚠️ **Si el baseline ganó:**
- Los hiperparámetros originales ya estaban bien calibrados
- El modelo baseline tiene buen balance entre complejidad y generalización
- **Acción**: Mantener baseline, considerar otras estrategias (ensemble, más datos)

### 🎯 **Próximos pasos recomendados:**
1. Validar el modelo ganador con datos independientes (test set)
2. Verificar estabilidad con diferentes random seeds
3. Considerar ensemble stacking (XGBoost + LightGBM + CatBoost)
4. Ajustar threshold de clasificación para maximizar AMS

---

---
## 🎓 **Resumen Final del Notebook**

### 📊 **Metodología Aplicada**

Este notebook implementó un proceso sistemático de optimización de hiperparámetros:

1. **Baseline**: Evaluación del modelo original con hiperparámetros por defecto
2. **Optimización Bayesiana**: Búsqueda inteligente con Optuna (50 trials)
3. **Validación Cruzada**: Cada configuración evaluada con CV 5-fold
4. **Selección Automática**: Mejor modelo guardado basado en AUC

### 🎯 **Ventajas de este Enfoque**

**✅ Optimización Bayesiana (Optuna) vs Grid Search:**
- **Más eficiente**: No prueba todas las combinaciones
- **Más inteligente**: Aprende de trials anteriores
- **Más rápido**: Converge a óptimo en menos iteraciones

**✅ Validación Cruzada Integrada:**
- Cada trial se valida con CV 5-fold
- Métricas más robustas y realistas
- Reduce riesgo de overfitting

### 💡 **Interpretación de Resultados**

**Si el modelo optimizado ganó por >2%:**
- ✅ Optimización exitosa
- ✅ Vale la pena usar hiperparámetros optimizados
- 🎯 Acción: Implementar en producción

**Si la mejora es 0.5-2%:**
- ⚠️ Mejora marginal
- ⚠️ Puede no generalizarse a datos nuevos
- 🎯 Acción: Validar con test set independiente

**Si el baseline ganó:**
- ✅ Hiperparámetros originales ya estaban bien calibrados
- ✅ Modelo robusto sin necesidad de ajustes
- 🎯 Acción: Mantener baseline, explorar otras estrategias

### 🔄 **Pipeline Sugerido para Producción**

```python
# 1. Cargar nuevo batch de datos
df_new = load_new_data()

# 2. Aplicar mismo feature engineering
df_new = add_feature_engineering(df_new)

# 3. Cargar modelo optimizado
model = joblib.load('models/best_model_optimized.pkl')

# 4. Predecir
predictions = model.predict_proba(df_new[current_features])[:, 1]

# 5. Aplicar threshold optimizado (si disponible)
classified = (predictions >= optimal_threshold).astype(int)

# 6. Monitorear drift
check_distribution_drift(df_new, df_train)
```

### 📈 **Métricas de Éxito**

**Proyecto Académico:**
- ✅ Demostrar comprensión de optimización bayesiana
- ✅ Aplicar validación cruzada correctamente
- ✅ Documentar proceso y resultados

**Producción CERN:**
- ✅ Validar con datos Run 2/Run 3 independientes
- ✅ Estabilidad en diferentes condiciones de detector
- ✅ Monitoreo continuo de performance

---

---
## 💾 **Paso 5: Guardar Mejor Modelo**

In [ ]:

+ç# Seleccionar mejor modelo basado en AUC
if auc_optimized > auc_baseline:
    print("✅ Guardando modelo OPTIMIZADO (mejor AUC)")
    final_model = best_model
    model_name = "best_model_optimized.pkl"
    is_optimized = True
else:
    print("✅ Guardando modelo BASELINE (mejor AUC)")
    final_model = model_actual
    model_name = "best_model_baseline.pkl"
    is_optimized = False

# Guardar modelo ganador
joblib.dump(final_model, f"../models/{model_name}")

# Guardar hiperparámetros del modelo optimizado
with open("../models/best_hyperparams.json", "w") as f:
    json.dump(study_xgb.best_params, f, indent=2)

print("✅ Modelo mejorado guardado:")
print("   📁 models/best_model_optimized.pkl")
print("   📁 models/enhanced_features.json")
print("   📁 models/best_hyperparams.json")

---
## 📊 **Paso 6: Comparación Visual (Curvas ROC)**

In [ ]:
from sklearn.metrics import roc_curve

# Calcular curvas ROC
fpr_base, tpr_base, _ = roc_curve(y, y_pred)
fpr_opt, tpr_opt, _ = roc_curve(y, y_pred_optimized)

# Visualizar
plt.figure(figsize=(10, 6))
plt.plot(fpr_base, tpr_base, label=f'Baseline (AUC={auc_baseline:.4f})', linewidth=2, color='blue')
plt.plot(fpr_opt, tpr_opt, label=f'Optimizado (AUC={auc_optimized:.4f})', linewidth=2, color='red')
plt.plot([0,1], [0,1], '--', color='gray', alpha=0.5, label='Random')

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Comparación ROC: Baseline vs Optimizado', fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Calcular ganancia en TPR a diferentes puntos de FPR
fpr_points = [0.01, 0.05, 0.10, 0.20]
print("\n📈 Ganancia en TPR (True Positive Rate) a diferentes FPR:")
print("-" * 60)
for fpr_val in fpr_points:
    idx_base = np.argmin(np.abs(fpr_base - fpr_val))
    idx_opt = np.argmin(np.abs(fpr_opt - fpr_val))
    tpr_gain = (tpr_opt[idx_opt] - tpr_base[idx_base]) * 100
    print(f"   FPR @ {fpr_val:.2f}: {tpr_gain:+.2f}% {'✅' if tpr_gain > 0 else '⚠️'}")

---
## 📝 **Paso 7: Conclusiones Finales**

### ✅ **Logros de este notebook:**

1. **Optimización Bayesiana**: Exploración sistemática de 50 combinaciones de hiperparámetros
2. **Validación Cruzada**: Cada trial evaluado con CV 5-fold para robustez
3. **Comparación Objetiva**: Métricas múltiples (AUC, Accuracy, F1, AMS)
4. **Selección Automática**: Mejor modelo guardado automáticamente

### 🎯 **Estrategias futuras (si necesitas más mejora):**

1. **Ensemble Stacking**: 
   - Combinar XGBoost + LightGBM + CatBoost
   - Meta-learner para predicción final
   - Ganancia típica: +1-3% AUC

2. **Calibración de Probabilidades**:
   - Platt scaling o isotonic regression
   - Mejora interpretabilidad de probabilidades
   - Útil para decisiones basadas en umbral

3. **Threshold Optimization**:
   - Ajustar punto de corte específicamente para maximizar AMS
   - Importante en física de partículas
   - Balance señal/fondo óptimo

4. **Feature Engineering Adicional**:
   - Variables cinemáticas de orden superior
   - Interacciones entre features
   - Transformaciones no lineales

5. **Data Augmentation**:
   - SMOTE para balance de clases
   - Bootstrapping de eventos
   - Simulación MC adicional

6. **Deep Learning**:
   - Red neuronal feedforward
   - Embeddings de features categóricas
   - Dropout para regularización

### 📚 **Referencias útiles:**

- **Optuna Documentation**: https://optuna.org/
- **XGBoost Parameter Tuning**: https://xgboost.readthedocs.io/en/stable/parameter.html
- **ATLAS ML Guidelines**: https://twiki.cern.ch/twiki/bin/view/AtlasPublic/MachineLearning
- **Higgs Kaggle Competition**: https://www.kaggle.com/c/higgs-boson

---